# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from workflow_widgets import WorkflowWidget, WorkerPoolWidget, TailWidget
from droplet_workflow import droplet_wf
from example_workflow import example_wf
from aux_widgets import QueueWidget, NEWTAuthWidget, SSHAuthWidget, SSHTerminal, Space
import workflow_objects as kale

import ipywidgets as ipw
import time
from concurrent.futures import ThreadPoolExecutor
import traitlets

In [3]:
import IPython.display as disp

# Define simple workflow

In [4]:
example_workflow = kale.Workflow(name='example')
for i in range(10):
    example_workflow.add_task(
        kale.CommandLineTask(
            name='echo_test_{num}',
            command='echo "Test #{num}" >> {out_file} &&',
            output_files=["{out_file}"],
            params=dict(
                num=i,
                out_file="output.txt"
            )
        )
    )

# Notes:
- Cluster:
    - Terminal should not start if not authenticated (status area)
    - Implement status area for SSHAuthWidget & SSHTerminal
- Worker Pool:
    - Location list should dynamically update, but class attributes can't be traitlets

# Connect to Cluster


In [5]:
ssh = SSHAuthWidget(host='cori.nersc.gov', username='oevans')
ssh

A Jupyter Widget

In [6]:
t = SSHTerminal(ssh, width=800)
t

A Jupyter Widget

# Create Worker Pool


In [7]:
wpw = WorkerPoolWidget()
wpw

A Jupyter Widget

# Visualize & Run Workflow

In [8]:
WorkflowWidget(droplet_wf, wpw)

A Jupyter Widget

In [9]:
WorkflowWidget(example_wf, wpw)

A Jupyter Widget

In [10]:
ww = WorkflowWidget(workflow=droplet_wf, worker_pool_widget=wpw)
ww

A Jupyter Widget

# Observe Output

In [11]:
t = TailWidget('out.txt')
t

A Jupyter Widget

# Connect to NEWT

In [12]:
a = NEWTAuthWidget()
a

A Jupyter Widget

In [13]:
q = QueueWidget(a)
q

A Jupyter Widget